In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('lpor_explorer.csv')

# One-hot encode columns in columns with 2 categories
df = pd.get_dummies(df, columns=['Escola', 'Sexo', 'TP_Moradia', 'Tamanho_Familia', 'Situacao_Pais', 'Apoio_Educacao_Extra', 'Apoio_Educacao_Pais', 'Aulas_Particulares', 'Tem_Internet', 'Quer_Fazer_Graduacao', 'Frequentou_Creche', 'Ativ_Extracurricular', 'Esta_Namorando'])
# One-hot encode columns in columns with more than 2 categories
df = pd.get_dummies(df, columns=['Motivo_Escolha_Escolar', 'Responsavel_Legal', 'Trabalho_Mae', 'Trabalho_Pai'])
# Drop columns: 'Frequentou_Creche_Não', 'Escola_Mousinho da Silveira' and 11 other columns
df = df.drop(columns=['Trabalho_Mae_other', 'Trabalho_Pai_other', 'Responsavel_Legal_Outro', 'Motivo_Escolha_Escolar_Outro', 'Frequentou_Creche_Não', 'Escola_Mousinho da Silveira', 'Sexo_Mulher', 'TP_Moradia_Rural', 'Tamanho_Familia_Até 3', 'Situacao_Pais_Morando Juntos', 'Apoio_Educacao_Extra_Não', 'Apoio_Educacao_Pais_Não', 'Aulas_Particulares_Não', 'Tem_Internet_Não', 'Quer_Fazer_Graduacao_Não', 'Ativ_Extracurricular_Não', 'Esta_Namorando_Não'])
df.columns = df.columns.str.replace('_Sim', '')    

# Put numbers instead categories in a column
def encoding(column, categories):
    for i, category in enumerate(categories):
        column = column.replace(category, i)
    df[column.name] = column

encoding(df.Estado_Saude, ['Muito Ruim', 'Ruim', 'Razoavel', 'Bom', 'Muito Bom'])
encoding(df.Boa_Convivencia_Familia, ['Muito Ruim', 'Ruim', 'Razoavel', 'Bom', 'Excelente']) 
encoding(df.Tempo_Estudo_Semanal, ['Até 2h,', '2 a 5h', '5 a 10h', 'Mais de 10h'])
encoding(df.Tempo_ida_Escola, ['Até 15 min', '15 a 30 min', '30 min a 1h', 'Mais de 1h'])
encoding(df.Educacao_Pai, ['Nenhuma', 'Ensino Fundamental 1', 'Ensino Fundamental 2', 'Ensino Médio', 'Ensino Superior'])
encoding(df.Educacao_Mae, ['Nenhuma', 'Ensino Fundamental 1', 'Ensino Fundamental 2', 'Ensino Médio', 'Ensino Superior'])
cat = ['Muito Baixo', 'Baixo', 'Moderado', 'Alto', 'Muito Alto']
encoding(df.Tempo_Livre_Apos_Escola, cat)
encoding(df.Tempo_com_Amigos, cat)
encoding(df.Alcool_Dia_Util, cat)
encoding(df.Alcool_Fim_Semana, cat)

# Replace both grades with the average of the two semesters
df['Media'] = df. Nota_1Semestre + df.Nota_2Semestre / 2
df = df.drop(columns=['Nota_1Semestre', 'Nota_2Semestre']) 

###############################################################################
target = ['Boa_Convivencia_Familia']

# Split features and target
X = df.drop(columns=target)
y = df[target].values.ravel()

# Normalization of DF
X = pd.DataFrame(MinMaxScaler().fit_transform(X), columns=X.columns)

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###############################################################################
def ML(model, modelName):
    # Model train
    model.fit(X_train, y_train)

    # Model evaluation
    print(modelName)
    print(f'Train: {model.score(X_train, y_train):.2%}')
    print(f'Test: {model.score(X_test, y_test):.2%}')
    print('')

# Caso Boa_Convivencia_Familia

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

ML(SVC(), 'SVC')
ML(RandomForestClassifier(), 'RandomForestClassifier')
ML(KNeighborsClassifier(), 'KNeighborsClassifier')
ML(XGBClassifier(), 'XGBClassifier')
ML(DecisionTreeClassifier(), 'DecisionTreeClassifier')
ML(AdaBoostClassifier(), 'AdaBoostClassifier')
ML(GaussianNB(), 'GaussianNB')

SVC
Train: 53.56%
Test: 48.46%

RandomForestClassifier
Train: 100.00%
Test: 45.38%

KNeighborsClassifier
Train: 58.19%
Test: 43.08%

XGBClassifier
Train: 100.00%
Test: 41.54%

DecisionTreeClassifier
Train: 100.00%
Test: 32.31%

AdaBoostClassifier
Train: 46.63%
Test: 32.31%

GaussianNB
Train: 20.62%
Test: 15.38%



In [3]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

ML(LogisticRegression(), 'LogisticRegression')
ML(LGBMClassifier(), 'LGBMClassifier')

C:\Users\jvict\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
Train: 56.26%
Test: 40.00%

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000230 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 156
[LightGBM] [Info] Number of data points in the train set: 519, number of used features: 37
[LightGBM] [Info] Start training from score -3.307465
[LightGBM] [Info] Start training from score -3.160861
[LightGBM] [Info] Start training from score -1.895195
[LightGBM] [Info] Start training from score -0.710640
[LightGBM] [Info] Start training from score -1.275170
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score


# Criar uma rede neural simples com TensorFlow/Keras
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
17/17 [==============================] - 1s 12ms/step - loss: -1.3731 - accuracy: 0.0424 - val_loss: -4.6604 - val_accuracy: 0.0538
Epoch 2/100
17/17 [==============================] - 0s 3ms/step - loss: -8.8863 - accuracy: 0.0424 - val_loss: -16.4933 - val_accuracy: 0.0538
Epoch 3/100
17/17 [==============================] - 0s 3ms/step - loss: -27.9789 - accuracy: 0.0424 - val_loss: -47.4698 - val_accuracy: 0.0538
Epoch 4/100
17/17 [==============================] - 0s 3ms/step - loss: -75.0929 - accuracy: 0.0424 - val_loss: -118.0582 - val_accuracy: 0.0538
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: -173.6558 - accuracy: 0.0424 - val_loss: -254.6852 - val_accuracy: 0.0538
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: -351.1333 - accuracy: 0.0424 - val_loss: -484.6669 - val_accuracy: 0.0538
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: -640.9003 - accuracy: 0.0424 - val_loss: -848.3677 - v